In [1]:
from combined_player import player_minutes_value
from html_scraper import db
from mongo_to_db import create_master_df

In [2]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_columns', 50)

# Dataframe of Players

In [4]:
games = db.games.find()
players = db.players.find()

In [5]:
final_df = player_minutes_value(games, players)

In [6]:
final_df.head()

,player_id,player_name,club,birthday,age,foot,squad_num,height,total_minutes_played,transfer_value(sterlings),transfer_value(USD)
0,19132.0,Alexander Domínguez,Squad Club Atlético Vélez Sarsfield,"Jun 5, 1987 (31)",31,right,22.0,"1,95 m",1080.000000,£1.58m,2.01
1,27987.0,Lucas Hoyos,Squad Club Atlético Vélez Sarsfield,"Apr 29, 1989 (29)",29,right,12.0,"1,83 m",90.000000,£135k,0.17
2,30896.0,Gastón Díaz,Squad Club Atlético Vélez Sarsfield,"Mar 13, 1988 (30)",30,right,24.0,"1,75 m",896.483333,£450k,0.57
3,94225.0,Joaquín Laso,Squad Club Atlético Vélez Sarsfield,"Jul 4, 1990 (28)",28,right,6.0,"1,85 m",990.000000,£270k,0.34
4,99779.0,Luis Abram,Squad Club Atlético Vélez Sarsfield,"Feb 27, 1996 (22)",22,left,29.0,"1,80 m",1080.000000,£405k,0.51


# Shot Data Dataframe

In [7]:
games = db.games.find()
shots_df = create_master_df(games)

/Users/david/galvanize/super_liga_xg/mongo_to_db.py:69: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  final_df = pd.concat([attach_to_df, df], axis=0, ignore_index=True)
/Users/david/galvanize/super_liga_xg/dataframe_cleaner.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_df['is_goal'] = input_df.loc[:, 'shot_type'].isin([9, 11, 12, 13]).astype(int)


In [8]:
shots_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0.0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0.0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0.0


# Model Training

In [9]:
from model_prep import create_training_df, create_test_df, create_xG_df, create_summed_xG_df, create_test_min_df, merged_dataframes

In [40]:
train_data, train_y, indices, hold_test = create_training_df(shots_df)
test_data, test_y, indices1, holdout, test = create_test_df(shots_df, hold_test)

In [41]:
train_data.head()

,player_id,shot_distance,shot_angle,assisted_shot,is_penalty_attempt
0,99779.0,6.838786,12.755990,1.0,0.0
1,60730.0,21.747333,11.242763,0.0,0.0
2,177285.0,12.000000,0.000000,0.0,1.0
3,185745.0,16.803407,59.849102,1.0,0.0
4,77892.0,26.220122,32.065607,0.0,0.0


In [42]:
train_y.head()

0    1.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: is_goal, dtype: float64

In [43]:
test_data.head()

,player_id,shot_distance,shot_angle,assisted_shot,is_penalty_attempt
48,68341.0,19.348478,36.467164,1.0,0.0
49,45616.0,25.331190,15.243216,0.0,0.0
50,71946.0,18.212551,42.953004,1.0,0.0
51,40397.0,15.043198,-12.789072,1.0,0.0
52,163302.0,24.483243,-15.784670,0.0,0.0


In [44]:
test_y.head()

48    0.0
49    0.0
50    0.0
51    0.0
52    0.0
Name: is_goal, dtype: float64

In [45]:
print(len(holdout))
#holdout is game_id where as the other two are shot events
print(len(train_data))
len(test_data)

54
1240


677

holdout is 54 games to train on again

training data is 1240 shots - rougly 60 games...

test data is 677 shots - roughly 34 games...

In [47]:
1240 / 20

62.0

# Running XGBoost, RF, Gradient Boosting, Ensemble

In [48]:
from sklearn.metrics import log_loss
from sklearn.ensemble import  RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [99]:
columns_to_predict_on = ['shot_distance', 'shot_angle', 'assisted_shot', 'is_penalty_attempt']

In [102]:
xgboost_model = XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 500 )
gradient_boost_model = GradientBoostingClassifier(learning_rate=0.01, max_depth=6, max_features='log2', min_samples_leaf=4, n_estimators=500, subsample=0.25)
random_forest_model = RandomForestClassifier(n_estimators=300, max_depth=3, verbose=1)

xgboost_model.fit(train_data[columns_to_predict_on], train_y)
gradient_boost_model.fit(train_data[columns_to_predict_on], train_y)
random_forest_model.fit(train_data[columns_to_predict_on], train_y)

p_random_forest = random_forest_model.predict_proba(test_data[columns_to_predict_on])
p_gradient_boost =  gradient_boost_model.predict_proba(test_data[columns_to_predict_on])
p_xgboost = xgboost_model.predict_proba(test_data[columns_to_predict_on])

#need to concat these two arrays somehow...
ensemble_p_0 = (p_random_forest[:,0] + p_gradient_boost[:,0] + p_xgboost[:,0])/3
ensemble_p_1 = (p_random_forest[:,1] + p_gradient_boost[:,1] + p_xgboost[:,1])/3
# ensemble_p = np.concatenate(ensemble_p_0, ensemble_p_1, axis=0)

random_forest_ll = log_loss(test_y, p_random_forest)
gradient_boost_ll = log_loss(test_y, p_gradient_boost)
xgboost_ll = log_loss(test_y, p_xgboost)
# ensemble_ll = log_loss(test_y, ensemble_p)
                                     

print("Ensemble Log Loss " + str(ensemble_ll))
print("Gradient Boost Log Loss " + str(gradient_boost_ll))
print("Random Forest Log Loss " + str(random_forest_ll))
print("XGBoost Log Loss " + str(xgboost_ll))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.32259131196059976
Gradient Boost Log Loss 0.3418100592045082
Random Forest Log Loss 0.32274524198829535
XGBoost Log Loss 0.31932849715157047


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.1s finished


# xG and minutes for various models

In [103]:
rf_xg = create_xG_df(test_data, test_y, p_random_forest)
rf_contributions = create_summed_xG_df(rf_xg)

gb_xg = create_xG_df(test_data, test_y, p_gradient_boost)
gb_contributions = create_summed_xG_df(gb_xg)

xgb_xg = create_xG_df(test_data, test_y, p_xgboost)
xgb_contributions = create_summed_xG_df(xgb_xg)

# ensem_xg = create_xG_df(test_data, test_y, ensemble_p)
# ensem_contributions = create_summed_xG_df(ensem_xg)

In [104]:
rf_contributions.head(20)

,player_id,total_xG,total_xA,total_xG+xA,goals
0,93853.0,2.45,1.48,3.93,2.0
1,101.0,1.84,0.28,2.12,2.0
2,47913.0,1.57,0.20,1.77,2.0
3,20195.0,1.41,0.00,1.41,1.0
4,17217.0,1.28,0.32,1.60,1.0
5,68341.0,1.17,0.63,1.80,2.0
6,203.0,1.12,0.68,1.80,1.0
7,16081.0,1.10,0.00,1.10,1.0
8,58422.0,1.07,0.00,1.07,1.0
9,98292.0,1.06,0.30,1.36,2.0


In [105]:
gb_contributions.head(20)

,player_id,total_xG,total_xA,total_xG+xA,goals
0,93853.0,2.10,1.13,3.23,2.0
1,17217.0,2.05,0.54,2.59,1.0
2,101.0,1.71,0.16,1.87,2.0
3,47913.0,1.60,0.08,1.68,2.0
4,172675.0,1.52,0.00,1.52,1.0
5,68341.0,1.50,0.83,2.33,2.0
6,70297.0,1.43,0.00,1.43,1.0
7,20195.0,1.39,0.00,1.39,1.0
8,150346.0,1.30,0.44,1.74,1.0
9,203.0,1.26,0.48,1.74,1.0


In [106]:
xgb_contributions.head(20)

,player_id,total_xG,total_xA,total_xG+xA,goals
0,93853.0,2.80,1.60,4.40,2.0
1,101.0,1.90,0.26,2.16,2.0
2,47913.0,1.63,0.18,1.81,2.0
3,17217.0,1.61,0.42,2.03,1.0
4,20195.0,1.51,0.00,1.51,1.0
5,98292.0,1.22,0.32,1.54,2.0
6,150346.0,1.22,0.56,1.78,1.0
7,68341.0,1.17,0.73,1.90,2.0
8,203.0,1.15,0.70,1.85,1.0
9,70297.0,1.15,0.00,1.15,1.0
